In [1]:
import pandas as pd
from sec_api import QueryApi
from sec_api import ExtractorApi


extractorApi = ExtractorApi(API_KEY)
queryApi = QueryApi(API_KEY)

def getCompany10ks(ticker):
    query = {
      "query": "ticker:"+ticker+" AND filedAt:[2023-07-29 TO 2024-07-29] AND formType:\"10-k\"",
      "from": "0",
      "size": "10",
      "sort": [{ "filedAt": { "order": "desc" } }]
    }
    response = queryApi.get_filings(query)
    return response['filings'][0]['documentFormatFiles'][0]['documentUrl']

def get10ktext(url):
    dict10k = {}
    section_items = ['1', '1A', '1B', '1C', '2', '3', '4', '5', '6', '7', '7A', '8', '9', '9A', '9B', '10', '11', '12', '13', '14', '15' ]
    text = " "
    for item in section_items:
        section_text = extractorApi.get_section(url, item, "text")
        dict10k[item] = section_text
    return dict10k
    
def getTables(url):
    tablesDict = {}
    section_items = ['1', '1A', '1B', '1C', '2', '3', '4', '5', '6', '7', '7A', '8', '9', '9A', '9B', '10', '11', '12', '13', '14', '15' ]
    for item in section_items:
        section_html = extractorApi.get_section(url, item, "html")
        try:
            tablesDF = pd.read_html(section_html)
            tablesDict[item] = tablesDF
        except ValueError:  # Catch the error raised when no tables are found
            tablesDict[item] = None  # or [] if you prefer an empty list
            print(f"No tables found for section {item}.")
    return tablesDict

def getCompanyData(ticker):
    url = getCompany10ks(ticker)
    textFiling = get10ktext(url)
    tables = getTables(url)
    return textFiling, tables

In [5]:
appleText, appleTables = getCompanyData("AAPL")

C:\Users\Willg\AppData\Local\Temp\ipykernel_18308\1030912514.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablesDF = pd.read_html(section_html)


No tables found for section 1.
No tables found for section 1A.
No tables found for section 1B.
No tables found for section 1C.
No tables found for section 2.
No tables found for section 3.
No tables found for section 4.


C:\Users\Willg\AppData\Local\Temp\ipykernel_18308\1030912514.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablesDF = pd.read_html(section_html)


No tables found for section 6.


C:\Users\Willg\AppData\Local\Temp\ipykernel_18308\1030912514.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablesDF = pd.read_html(section_html)
C:\Users\Willg\AppData\Local\Temp\ipykernel_18308\1030912514.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablesDF = pd.read_html(section_html)
C:\Users\Willg\AppData\Local\Temp\ipykernel_18308\1030912514.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tablesDF = pd.read_html(section_html)


No tables found for section 9.
No tables found for section 9A.
No tables found for section 9B.
No tables found for section 10.
No tables found for section 11.
No tables found for section 12.
No tables found for section 13.
No tables found for section 14.


In [3]:
from sec_api import XbrlApi

xbrlApi = XbrlApi("9a658e0f3e03d9f882ff1529631d3f2120986bafaa496b0c3a66856ccbbb19be")

# 10-K HTM File URL example
htm_url="https://www.sec.gov/Archives/edgar/data/320193/000032019320000096/aapl-20200926.htm"
xbrl_json = xbrlApi.xbrl_to_json(htm_url=htm_url)

In [9]:
import pandas as pd
from sec_api import QueryApi
from sec_api import ExtractorApi
from sec_api import XbrlApi
API_KEY = "9a658e0f3e03d9f882ff1529631d3f2120986bafaa496b0c3a66856ccbbb19be"
extractorApi = ExtractorApi(API_KEY)
queryApi = QueryApi(API_KEY)
xbrlApi = XbrlApi(API_KEY)


def companyDataJson(ticker):
    url = getCompany10ks(ticker)
    json = xbrlApi.xbrl_to_json(htm_url=url)
    return json
    
# convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement.T 
    
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                    
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheet = pd.DataFrame(balance_sheet_store)
    # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
    return balance_sheet.T

In [11]:
income_statement = get_income_statement(apple)
balance_sheet = get_balance_sheet(apple)

NameError: name 'apple' is not defined